In [2]:
import torch
import torchvision
from torchvision import transforms
from torch.utils.data import DataLoader, Dataset
import torch.nn as nn
import torch.optim as optim
import torch.optim.lr_scheduler as lr_sched
import torch.utils.data as data

In [3]:
if torch.cuda.is_available():
    device = "cuda"
else:
    device = "cpu"

In [4]:
"""
Do grzyb/nie grzyb - transform_raw_1
Do klasyfikacji - transform_raw_2 (zachowuje więcej szczegółów)
"""
transform_raw_1 = transforms.Compose([
    transforms.Resize([96, 96]),
    transforms.ToTensor(),
     transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])]
)

transform_raw_2 = transforms.Compose([
    transforms.Resize([250, 250]),
    transforms.ToTensor(),
     transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])]
)

In [5]:
dataset = torchvision.datasets.ImageFolder("DS_MASHROOM_OR_NOT", transform=transform_raw_1)
len(dataset)

99249

In [6]:
seed = 123
torch.manual_seed(seed)

In [7]:
train_size = 95000
val_size = len(dataset) - train_size

train_ds, val_ds = data.random_split(dataset, [train_size, val_size])

In [8]:
batch_size = 500

train_loader = DataLoader(train_ds, batch_size=batch_size, shuffle=True, pin_memory=True, num_workers=1)
val_loader = DataLoader(val_ds, batch_size=batch_size, pin_memory=True, num_workers=1)

In [9]:
class RestPercepton_block(nn.Module):
    """
    Base class for percepton block with residual connection (no pre-activation and BN before conv)

    in_channels - num. channles into block
    out_channels - num. concatenated channles out from block
    conv_size_in - list of num. of channels into 1,3 and 5 conv (respectively)
    conv_size_out - list of num. of channels going out from 1,3,5 conv (-||-)
    stride, padding - list with stride and padding values for 1, 3, 5 conv respectively
    change_depth_pool - change depth for pooling. By default "False"(no change), if used must be int (out depth from pool section)
    """


    #TODO add batch normalization and activation functions after conv
    def __init__(self, in_channels, out_channels,
                conv_size_in:list, conv_size_out:list,
                stride:list=[1,1,2], padding:list=[0, 1, 2],
                change_depth_pool=False):

        # checking if dim are correct
        if(out_channels != sum(conv_size_out)):
            raise ValueError(
                "Sum of out channels of the block must be equal to sum of out channels of convs inside the block"
            )

        super().__init__()
        self.in_channels = in_channels
        self.out_channels = out_channels

        # conv 1x1
        self.conv1 = nn.Sequential(
            nn.Conv2d(conv_size_in[0], conv_size_out[0], kernel_size=1, stride=stride[0], padding=padding[0]),
            nn.BatchNorm2d(conv_size_out[0]),
            nn.ReLU()
        )

        # conv 3x3
        self.conv3 = nn.Sequential(
            nn.Conv2d(conv_size_in[1], conv_size_out[1], kernel_size=3, stride=stride[1], padding=padding[1]),
            nn.BatchNorm2d(conv_size_out[1]),
            nn.ReLU()
        )

        if(conv_size_in[1] != in_channels): # change depth if intercepton's input has different depth size than 3x3 conv_in_depth
            self.conv3 = nn.Sequential(
                nn.Conv2d(in_channels, conv_size_in[1], kernel_size=1, padding=0, stride=1),
                self.conv3
        )

       # conv 5x5
        self.conv5 = nn.Sequential(
            nn.Conv2d(conv_size_in[2], conv_size_out[2], kernel_size=5, stride=stride[2], padding=padding[2]),
            nn.BatchNorm2d(conv_size_out[2]),
            nn.ReLU()
        )

        if(conv_size_in[1] != in_channels): # change depth if intercepton's input has different depth size than 5x5 conv_in_depth
            self.conv5 = nn.Sequential(
                nn.Conv2d(in_channels, conv_size_in[2], kernel_size=1, padding=0, stride=1),
                self.conv5
        )

        # max pool 3x3
        if(not change_depth_pool):
           self.pool = nn.MaxPool2d(kernel_size=3, stride=2)
        else:
            self.pool = nn.Sequential(
                nn.Conv2d(in_channels, change_depth_pool, kernel_size=1, padding=0, stride=1),
                self.depth_change(in_channels, change_depth_pool),
                nn.BatchNorm2d(change_depth_pool),
                nn.ReLU()
            )


    def forward(self, x):

        conv1 = self.conv1(x)
        conv3 = self.conv3(x)
        conv5 = self.conv5(x)
        pool = self.pool(x)

        if(self.in_channels != self.outchannels):
            rest_conv = nn.Conv2d(self.in_channels, self.out_channels, kernel_size=1, stride=1)
            residual = rest_conv(x)
        else:
            residual = x


        return(torch.cat([conv1, conv3, conv5, pool], dim=1) + residual)

In [ ]:
class RestGoogleNet_BinaryClasificator(nn.Module):
    """
    Input - nx96x96 photo
    """
    def __init__(self, in_channels):
        super().__init__()


        self.Intercepton1 = RestPercepton_block(in_channels=32, out_channels=64, conv_size_in=[])
        self.Intercepton2 = RestPercepton_block()
        self.Intercepton3 = RestPercepton_block()
        self.Intercepton4 = RestPercepton_block()


        self.network = nn.Sequential(

            nn.Conv2d(in_channels, out_channels=16, kernel_size=5, stride=1, padding=2),
            nn.BatchNorm2d(16),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2), # 16x48x48

            nn.Conv2d(in_channels=16, out_channels=32, kernel_size=5, stride=1, padding=2),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2), # 32x24x24

            self.Intercepton1(),
            self.Intercepton2,
            nn.MaxPool2d(kernel_size=2, stride=2),

            self.Intercepton3,
            nn.AvgPool2d(kernel_size=2, stride=2),


            # fully connected
            nn.Dropout(0.5),
            nn.Linear(1234, 1)
        )